In [1]:
# use tools to get existing annotationos;
# main reason is that both species/reaction class and recom class
# need getExistingAnnotation
import collections
import compress_pickle
import editdistance
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import re
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'

# dir for alternative methods for species
ALT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/alt_methods_species_data'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

/Users/woosubs/anaconda3/envs/pyo2/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/woosubs/anaconda3/envs/pyo2/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
# # under tools
# def extractExistingSpeciesAnnotation(inp_model, qualifier=cn.CHEBI):
#   """
#   Get existing annotation of species
#   that contains ChEBI terms
#   """
#   exist_raw = {val.getId():tools.getQualifierFromString(val.getAnnotationString(), qualifier) \
#                for val in model.getListOfSpecies()}
#   exist_filt = {val:exist_raw[val] for val in exist_raw.keys() \
#                 if exist_raw[val] is not None}
#   return exist_filt

In [22]:
one_biomd = 'BIOMD0000000248.xml'
one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)

reader = libsbml.SBMLReader()
document = reader.readSBML(one_fpath)
model = document.getModel()


# e-coli model
# document = reader.readSBML(ecoli_fpath)
# model = document.getModel()

In [4]:
# [tools.getOntologyFromString(val.getAnnotationString()) for val in model.getListOfReactions()]

In [5]:
reac_dict_raw_rhea = {r.getId():tools.getQualifierFromString(r.getAnnotationString(), cn.RHEA) \
                     for r in model.getListOfReactions()}
reac_dict_raw_filt_rhea = {k:reac_dict_raw_rhea[k] \
                           for k in reac_dict_raw_rhea.keys() \
                           if reac_dict_raw_rhea[k] is not None}
reac_dict_format_rhea = {k:[cn.RHEA_HEADER+val for val in reac_dict_raw_filt_rhea[k]] \
                           for k in reac_dict_raw_filt_rhea.keys()}
## MAP to BI_BI format, 

In [11]:
# it seems better to collect identifiers of a reactiono first,
# and combine them into a single identifier
one_rid = 'R_PFK'
one_r = model.getReaction(one_rid)
one_ranot = one_r.getAnnotationString()

exist_rheas = [cn.RHEA_HEADER+val for val in tools.getQualifierFromString(one_ranot, cn.RHEA)]
map_rhea_bis = [cn.REF_RHEA2BI[val] for val in exist_rheas if val in cn.REF_RHEA2BI.keys()]

exist_keggs = [val for val in tools.getQualifierFromString(one_ranot, cn.KEGG_REACTION)]
map_kegg2rhea = [cn.REF_KEGG2RHEA_BI[val] for val in exist_keggs if val in cn.REF_KEGG2RHEA_BI.keys()]

exist_ecs = [cn.EC_HEADER+val for val in tools.getQualifierFromString(one_ranot, cn.EC)]
map_ec2rhea = list(itertools.chain(*[cn.REF_EC2RHEA_BI[val] \
                                    for val in exist_ecs if val in cn.REF_EC2RHEA_BI.keys()]))

rhea_combined = list(set(map_rhea_bis + map_kegg2rhea + map_ec2rhea))

In [19]:
# def extractRheaFromAnnotationString(inp_str):
#   """
#   Extract Rhea from existing annotation string,
#   by directly extracting Rhea,
#   and converting from KEGGG and EC-Code. 
  
#   Parameters
#   ----------
#   inp_str: str
  
#   Returns
#   -------
#   list-str
#   """
#   exist_rheas = [cn.RHEA_HEADER+val for val in tools.getQualifierFromString(inp_str, cn.RHEA)]
#   map_rhea_bis = [cn.REF_RHEA2BI[val] for val in exist_rheas if val in cn.REF_RHEA2BI.keys()]

#   exist_keggs = [val for val in tools.getQualifierFromString(inp_str, cn.KEGG_REACTION)]
#   map_kegg2rhea = [cn.REF_KEGG2RHEA_BI[val] for val in exist_keggs if val in cn.REF_KEGG2RHEA_BI.keys()]

#   exist_ecs = [cn.EC_HEADER+val for val in tools.getQualifierFromString(inp_str, cn.EC)]
#   map_ec2rhea = list(itertools.chain(*[cn.REF_EC2RHEA_BI[val] \
#                                       for val in exist_ecs if val in cn.REF_EC2RHEA_BI.keys()]))

#   return list(set(map_rhea_bis + map_kegg2rhea + map_ec2rhea))

# def extractExistingReactionAnnotation(inp_model):
#   """
#   Get existing annotation of reactions in Rhea
#   in Bi-directional format (RHEA:10003, etc.)
#   This will extract annotations from three
#   knowledge resources:
#   1. Rhea (mapped to BI-format)
#   2. KEGG (kegg.reaction mapped to Rhea-BI)
#   3. EC-Number (or ec-code, mapped to list of Rhea-BIs)
  
#   Once they are mapped to a list of Rhea terms,
#   a list of unique Rhea-Bi terms will be filtered
#   and be assigned to exist_annotation of the 
#   reaction_annotation class instance.
  
#   Parameters
#   ----------
#   inp_model: libsbml.Model
  
#   Returns
#   -------
#   dict
#   """
#   exist_raw = {val.getId():extractRheaFromAnnotationString(val.getAnnotationString()) \
#                for val in model.getListOfReactions()}
#   exist_filt = {val:exist_raw[val] for val in exist_raw.keys() \
#                 if exist_raw[val]}
#   return exist_filt

In [20]:
cn.REF_RHEA2BI['RHEA:16112']

'RHEA:16112'

In [31]:
extractRheaFromAnnotationString(model.getReaction('CreatineKinase').getAnnotationString())

['RHEA:17160']

In [23]:
extractExistingReactionAnnotation(model)

{'CreatineKinase': ['RHEA:17160']}

In [30]:
model.getReaction('CreatineKinase').getAnnotationString()

'<annotation>\n  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">\n    <rdf:Description rdf:about="#metaid_0000084">\n      <bqbiol:isVersionOf>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/ec-code/2.7.3.2"/>\n          <rdf:li rdf:resource="http://identifiers.org/kegg.reaction/R01881"/>\n          <rdf:li rdf:resource="http://identifiers.org/go/GO:0004111"/>\n        </rdf:Bag>\n      </bqbiol:isVersionOf>\n    </rdf:Description>\n  </rdf:RDF>\n</annotation>'

In [26]:
anot_reactions = {val.getId():val.getAnnotationString() for val in model.getListOfReactions()}
print(anot_reactions['ATPase'])

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000022">
      <bqbiol:isVersionOf>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/ec-code/3.6.1.3"/>
          <rdf:li rdf:resource="http://identifiers.org/go/GO:0016887"/>
        </rdf:Bag>
      </bqbiol:isVersionOf>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [27]:
ec_codes = cn.REF_EC2RHEA_BI
'EC:3.6.1.3' in ec_codes

False

In [28]:
ec_codes

{'EC:3.5.1.50': ['RHEA:10003'],
 'EC:5.99.1.1': ['RHEA:10007'],
 'EC:1.5.3.6': ['RHEA:10015'],
 'EC:3.1.1.49': ['RHEA:10019'],
 'EC:1.5.1.10': ['RHEA:10023'],
 'EC:1.4.3.7': ['RHEA:10031'],
 'EC:1.4.3.15': ['RHEA:10031', 'RHEA:12515'],
 'EC:3.5.1.43': ['RHEA:10035'],
 'EC:1.1.3.38': ['RHEA:10039'],
 'EC:3.1.3.7': ['RHEA:10043'],
 'EC:1.1.1.245': ['RHEA:10047'],
 'EC:2.5.1.49': ['RHEA:10051', 'RHEA:27825'],
 'EC:1.1.1.345': ['RHEA:10055', 'RHEA:35646'],
 'EC:4.3.1.14': ['RHEA:10059'],
 'EC:2.3.1.34': ['RHEA:10063'],
 'EC:2.1.2.7': ['RHEA:10067'],
 'EC:2.4.1.53': ['RHEA:10071'],
 'EC:2.1.1.161': ['RHEA:10075'],
 'EC:3.2.1.108': ['RHEA:10079'],
 'EC:1.3.1.57': ['RHEA:10083'],
 'EC:1.13.11.3': ['RHEA:10087'],
 'EC:2.4.1.167': ['RHEA:10091'],
 'EC:3.1.3.34': ['RHEA:10095'],
 'EC:2.3.1.184': ['RHEA:10099'],
 'EC:3.1.1.13': ['RHEA:10103', 'RHEA:36406'],
 'EC:1.14.11.15': ['RHEA:10107'],
 'EC:1.1.1.248': ['RHEA:10111'],
 'EC:4.2.1.22': ['RHEA:10115'],
 'EC:1.1.1.44': ['RHEA:10119'],
 'EC:1.1.1